In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

sys.path.append('../..')
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency, metrics

import helper

np.random.seed(247)
tf.set_random_seed(247)

In [2]:
# dataset path# save path
results_path = utils.make_directory('../results', 'synthetic')
params_path = utils.make_directory(results_path, 'model_params')

# load dataset
data_path = '../data/synthetic_dataset.h5'
train, valid, test = helper.load_synthetic_dataset(data_path)

test_model = helper.load_synthetic_TF_models(data_path, dataset='test')
    
    
# get data shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = [None, train['targets'].shape[1]]

loading training data
loading cross-validation data
loading test data


In [3]:
tf.reset_default_graph()

model_name = 'cnn_2_50'
# model_name = 'cnn_25_4'

# load model parameters
genome_model = helper.import_model(model_name)
model_layers, optimization = genome_model.model(input_shape, output_shape)

# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization, supervised=True)
nnmodel.inspect_layers()

# create neural trainer
file_path = os.path.join(params_path, model_name)
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

# initialize session
sess = utils.initialize_session()

nntrainer.set_best_parameters(sess)

activations = nntrainer.get_activations(sess, test, layer='output')
threshold = nntrainer.get_activations(sess, test, layer=nnmodel.network.keys()[-2])

sess.close()

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: inputs
(?, 200, 1, 4)
layer2: conv1d_0
(?, 200, 1, 30)
layer3: conv1d_0_batch
(?, 200, 1, 30)
layer4: conv1d_0_active
(?, 200, 1, 30)
layer5: conv1d_0_maxpool
(?, 100, 1, 30)
layer6: conv1d_0_dropout
(?, 100, 1, 30)
layer7: conv1d_1
(?, 100, 1, 128)
layer8: conv1d_1_batch
(?, 100, 1, 128)
layer9: conv1d_1_active
(?, 100, 1, 128)
layer10: conv1d_1_maxpool
(?, 2, 1, 128)
layer11: conv1d_1_dropout
(?, 2, 1, 128)
layer12: dense_0
(?, 512)
layer13: dense_0_batch
(?, 512)
layer14: dense_0_active
(?, 512)
layer15: dense_0_dropout
(?, 512)
layer16: dense_1
(?, 12)
layer17: dense_1_bias
(?, 12)
layer18: output
(?, 12)
----------------------------------------------------------------------------
loading model from:  ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../res

In [5]:
save_path = utils.make_directory(results_path, 'saliency')

class_sum = np.sum(test['targets'], axis=1)

for class_index in range(12):
    
    right_index = np.where((test['targets'][:,class_index]==1) & 
                                   (class_sum == 1)
                                  )[0]

    X = test['inputs'][right_index]
    X_model = test_model[right_index]

    model_name = 'cnn_2_50'
    genome_model = helper.import_model(model_name)
    file_path = os.path.join(params_path, model_name)
    params = {'model_name': model_name, 
              'genome_model': genome_model.model,
              'input_shape': input_shape, 
              'output_shape': output_shape, 
              'model_path': file_path+'_best.ckpt',
             }
    guided_saliency = saliency.guided_backprop(X, layer='output', class_index=class_index, params=params)
    
    info = []
    for j, gs in enumerate(guided_saliency):
        X_saliency = np.squeeze(gs).T
        good_info, bad_info = helper.info_weighted_distance(X_saliency, X_model[j])
        info.append([good_info, bad_info])
    info = np.array(info)

    indices = np.where((info[:,0] > 0.97) & (info[:,1]<0.01))[0]
    if not indices.any():
        indices = np.where((info[:,0] > 0.95) & (info[:,1]<0.03))[0]
    if not indices.any():
        indices = np.where((info[:,0] > 0.9) & (info[:,1]<0.05))[0]
    indices = indices[np.argsort(info[indices, 0])][::-1]
    
    for index in indices[:5]:
        X_sample = np.expand_dims(X[index], axis=0)
        X_sample_model = X_model[index]


        model_name = 'cnn_25_4'
        genome_model = helper.import_model(model_name)
        file_path = os.path.join(params_path, model_name)
        params = {'model_name': model_name, 
                  'genome_model': genome_model.model,
                  'input_shape': input_shape, 
                  'output_shape': output_shape, 
                  'model_path': file_path+'_best.ckpt',
                 }
        shallow_guided_saliency = saliency.guided_backprop(X_sample, layer='output', class_index=class_index, params=params)


        fig = plt.figure(figsize=(50,5))
        plt.subplot(2,1,1)
        visualize.plot_pos_saliency(np.squeeze(shallow_guided_saliency).T, alphabet='dna',height=150, nt_width=50, norm_factor=3)
        plt.axis('on')
        plt.xticks([])
        plt.yticks([])
        plt.ylabel('CNN-25', fontsize=36)
        plt.subplot(2,1,2)
        visualize.plot_pos_saliency(np.squeeze(guided_saliency[index]).T, alphabet='dna',height=150, nt_width=50, norm_factor=3)
        plt.axis('on')
        plt.xticks([])
        plt.yticks([])
        plt.ylabel('CNN-2', fontsize=36)
        outfile = os.path.join(save_path, str(class_index)+'_comparison_'+str(index)+'.pdf')
        fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight') 

        fig = plt.figure(figsize=(50,3))
        visualize.plot_seq_pos_saliency(np.squeeze(X_sample).T, X_sample_model, alphabet='dna', nt_width=50, norm_factor=3)
        outfile = os.path.join(save_path, str(class_index)+'_model_'+str(index)+'.pdf')
        fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight') 


making directory: ../results/synthetic/saliency
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
INFO:tensorflow:Restoring parameters from ../results/synthetic/model_params/cnn_2_50_best.ckpt
IN